
# Notion API Beginner Tutorial


## Getting Started: Install Notion's Python SDK

In [1]:
from notion_client import Client
import pandas as pd

### API Key/Token

In [2]:
# I'm using this to load my Notion API token from a .env file.
from dotenv import load_dotenv
load_dotenv()

import os
try:
    NOTION_TOKEN = os.environ.get("NOTION_TOKEN")
except: 
    pass

# You can also just paste your token in here directly.
if NOTION_TOKEN is None:
    print("No token found. Please enter your Notion API token:")
    NOTION_TOKEN = input()

In [3]:
# Give the key to our notion client so we can begin making API calls
notion = Client(auth=NOTION_TOKEN)

## Familiarize Yourself with the SDK functions.

### *The SDK offers us some simple ways to interact with Notion's API.*

Some endpoints require a json object that would be the definition for creating new, or updating objects like Databases, Pages, or Page Blocks.

- https://developers.notion.com/reference/block
- https://developers.notion.com/reference/page
- https://developers.notion.com/reference/database


### Each endpoint function can be categorized into the following objects:

### 1. Search All of Notion
notion.search("Optional - search keywords")['results'] -> *Returns a list of all databases and pages.*

### 2. Databases
notion.databases.query(database_id)                     -> *Query/search a database's records, by database_id*

notion.databases.retrieve(database_id)                  -> *Retrieve information about a database, by database_id*

notion.databases.create(parent_id, database_object)     -> *Creates a new database*

notion.databases.update(database_id, database_object)   -> *Update a database*

### 3. Pages
notion.pages.create(parent_id, page_object)             -> Create a new page, by parent_id (database_id or page_id)

notion.pages.retrieve(page_id)                          -> Retrieve a page's details by page_id

notion.pages.update(page_id, page_object)               -> Update a page_id, requires a page object.

### 4. Users
notion.users.list()                                     -> *List all users*

notion.users.retrieve(user_id)                          -> *Retrieve a specific user, by user_id*

notion.users.me()                                       -> *Return the authenticated user*

### 5. Blocks (on a page)
notion.blocks.retrieve(block_id or page_id)             -> *Return the blocks from a page_id or block_id.

notion.blocks.update(block_id)                          -> *Update a block, by block_id. Requires a Block object.*

notion.blocks.delete(block_id)                          -> *Delete a block, by block_id*

### 6. Block Children
notion.blocks.children.append(block_id, child_object)   -> *Add blocks to an existing block_id. Requres a Block object.*

notion.blocks.children.list(block_id)                   -> *List the blocks that are children of a block, or page.*




## List All Databases and Pages with `notion.search()`

In [4]:
# Searching all pages and databases
search_results = notion.search()['results']

# Seperate the list into pages and databases
databases = [x for x in search_results if x['object'] == 'database']
pages = [x for x in search_results if x['object'] == 'page']

## Turning the API Response Into a Table Using `pd.DataFrame()`

In [5]:
# This function will help to convert into something SQL friendly.
def databases_to_table(databases):
  df = pd.DataFrame(databases)

  # Here we "un-nest" the json objects to obtain the value - not all columns need this treatment.
  # This step is not even necessary but does clean it up a little bit
  df['created_by'] = [x['id'] for x in df.created_by]
  df['last_edited_by'] = [x['id'] for x in df.last_edited_by]
  df['title'] = [x[0]['text']['content'] for x in df.title]
  df['parent'] = [x[x['type']] for x in df.parent]
  df['icon'] = [x['emoji'] if x is not None else None for x in df.icon] 
  df['description'] = [None if x == [] else x[0]['text']['content'] for x in df.description] 
  df['cover'] = [None if x == None else x[x['type']]['url'] for x in df.cover] 

  return df

df = databases_to_table(databases)
df

,object,id,cover,icon,created_time,created_by,last_edited_by,last_edited_time,title,description,is_inline,properties,parent,url,public_url,archived
0,database,d6463e42-f2aa-4dca-aa03-b36b7a293946,None,📐,2023-07-27T23:00:00.000Z,1099078b-21c5-446c-9a86-875533efa7c3,1099078b-21c5-446c-9a86-875533efa7c3,2023-08-05T20:33:00.000Z,Master Projects,A Database of projects. Each project can have ...,True,"{'📚 Master Resources': {'id': 'DSLa', 'name': ...",476a8eef-e8f5-40c5-97e1-e869296fe8c7,https://www.notion.so/d6463e42f2aa4dcaaa03b36b...,None,False
1,database,4288c73f-2fb1-4c91-aab3-4a1529d03d84,https://www.notion.so/images/page-cover/woodcu...,💩,2023-08-05T19:31:00.000Z,1099078b-21c5-446c-9a86-875533efa7c3,1099078b-21c5-446c-9a86-875533efa7c3,2023-08-05T20:30:00.000Z,Database: Morning Dump,A Description,False,"{'Created time': {'id': 'UA%3Br', 'name': 'Cre...",c8f6b474-0241-437a-b6cb-5305853e8e57,https://www.notion.so/4288c73f2fb14c91aab34a15...,None,False
2,database,07cc6456-7a46-4ae5-9897-8db47e6bc72a,https://www.notion.so/images/page-cover/rijksm...,🕝,2023-08-05T19:23:00.000Z,1099078b-21c5-446c-9a86-875533efa7c3,1099078b-21c5-446c-9a86-875533efa7c3,2023-08-05T20:29:00.000Z,Database: Meditation Journal,A description,False,"{'Name': {'id': 'title', 'name': 'Name', 'type...",eb84fff7-2b56-4315-bd6a-3bb3cb7f4bd9,https://www.notion.so/07cc64567a464ae598978db4...,None,False
3,database,24ad361a-95c2-4a79-a9d7-117c53a0656c,https://www.notion.so/images/page-cover/met_te...,💤,2023-08-05T19:19:00.000Z,1099078b-21c5-446c-9a86-875533efa7c3,1099078b-21c5-446c-9a86-875533efa7c3,2023-08-05T20:28:00.000Z,Database: Dream Journal,Dream Journal database description,False,"{'Created time': {'id': 'UA%3Br', 'name': 'Cre...",40d5ba88-2256-47d6-ac24-48f78dc4944d,https://www.notion.so/24ad361a95c24a79a9d7117c...,None,False
4,database,ae3cef49-742e-4dfc-910d-148ebc1d992b,None,🖊️,2023-07-27T23:01:00.000Z,1099078b-21c5-446c-9a86-875533efa7c3,1099078b-21c5-446c-9a86-875533efa7c3,2023-08-05T20:26:00.000Z,Master Journals,A daily tracker to start and end each day with.,True,"{'Created time': {'id': 'fXfM', 'name': 'Creat...",476a8eef-e8f5-40c5-97e1-e869296fe8c7,https://www.notion.so/ae3cef49742e4dfc910d148e...,None,False
5,database,cc186f3d-3bbe-4738-ab3f-a2065c03f3ff,None,✅,2023-07-27T23:00:00.000Z,1099078b-21c5-446c-9a86-875533efa7c3,1099078b-21c5-446c-9a86-875533efa7c3,2023-08-05T20:18:00.000Z,Master Tasks,A database of tasks. Each task is associated w...,True,"{'Success Criteria': {'id': 'iH%3FX', 'name': ...",476a8eef-e8f5-40c5-97e1-e869296fe8c7,https://www.notion.so/cc186f3d3bbe4738ab3fa206...,None,False
6,database,959a58e3-cd71-481d-bc3f-4d782b5ffbd8,None,🏆,2023-08-04T22:07:00.000Z,1099078b-21c5-446c-9a86-875533efa7c3,1099078b-21c5-446c-9a86-875533efa7c3,2023-08-05T20:18:00.000Z,Master Goals,Contains our current goals. Projects & tasks s...,True,"{'🌐 Master Areas': {'id': 'iI%3C%3C', 'name': ...",476a8eef-e8f5-40c5-97e1-e869296fe8c7,https://www.notion.so/959a58e3cd71481dbc3f4d78...,None,False
7,database,79e7cefe-a105-4aed-9438-2612aa3394e2,None,🌐,2023-07-27T23:01:00.000Z,1099078b-21c5-446c-9a86-875533efa7c3,1099078b-21c5-446c-9a86-875533efa7c3,2023-08-05T20:18:00.000Z,Master Areas,High-level areas of your life. Generally areas...,True,"{'🏆 Master Goals': {'id': '%3B%5E%3AX', 'name'...",476a8eef-e8f5-40c5-97e1-e869296fe8c7,https://www.notion.so/79e7cefea1054aed94382612...,None,False
8,database,8b195f93-1d48-4f3e-abbe-1c7185f6b4ca,None,📚,2023-07-27T23:01:00.000Z,1099078b-21c5-446c-9a86-875533efa7c3,1099078b-21c5-446c-9a86-875533efa7c3,2023-08-04T22:55:00.000Z,Master Resources,A database of helpful resources. Each record i...,True,"{'📐 Master Projects': {'id': 'BPyF', 'name': '...",476a8eef-e8f5-40c5-97e1-e869296fe8c7,https://www.notion.so/8b195f931d484f3eabbe1c71...,None,False


In [6]:
# High-level look at our new "Databases" table
print(f'Number of Databases: {len(databases)}.\n')
print(f'Number of Pages: {len(pages)}.\n')

Number of Databases: 9.

Number of Pages: 32.

